In [1]:
import pandas as pd
import os
import shutil

In [2]:
def list_all_folders(directory):
    try:
        # 使用 os.listdir 遍歷目錄
        folders = [name.replace('\\', '/') for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]
        return folders
    except Exception as e:
        print(f"發生錯誤：{e}")
        return []

In [3]:
def find_jpg_files(root_folder, foldername):
    df = pd.DataFrame()

    # 遍历目录
    folder = os.path.join(root_folder, foldername).replace('\\', '/')
    file_list = []
    for dirpath, _, filenames in os.walk(folder):
        for filename in filenames:
            if filename.lower().endswith('.webp'):
                full_path = os.path.join(dirpath, filename)
                file_list.append({'filename': filename, 'full_path': full_path, 'foldername':foldername})
    tmp_df = pd.DataFrame(file_list, columns=['filename', 'full_path', 'foldername'])
    df = pd.concat([df, tmp_df])
    print('')
    print(f'{foldername}: {tmp_df.shape}')

    df['full_path'] = df['full_path'].apply(lambda x: x.replace('\\', '/'))
    return df

In [4]:
def delete_file(files, path):
    file_list = files.strip().splitlines()
    success_count = 0
    failure_count = 0
    failed_files = []

    # 搜尋目錄中所有的檔案
    file_paths = []
    for root, _, filenames in os.walk(path):
        for filename in filenames:
            if filename in file_list:
                file_paths.append(os.path.join(root, filename))

    # 刪除檔案
    for full_path in file_paths:
        try:
            os.remove(full_path)
            success_count += 1
        except Exception as e:
            failure_count += 1
            failed_files.append((full_path, str(e)))

    print(f"成功刪除 {success_count} 個檔案")
    print(f"刪除失敗 {failure_count} 個檔案")
    if failed_files:
        print("失敗檔案列表：")
        for failed_file, error in failed_files:
            print(f"- {failed_file}: {error}")

In [5]:
def data_to_csv(df):
    output_dir = './data/test'
    os.makedirs(output_dir, exist_ok=True)  # 確保輸出目錄存在

    for folder in df['foldername'].unique():
        df_tmp = df[df['foldername'] == folder]
        # 按檔名排序（忽略大小寫）
        df_tmp = df_tmp.sort_values('filename', key=lambda x: x.str.lower()).reset_index(drop=True)

        # 計算總批次數
        batch_size = 200
        num_batches = (len(df_tmp) + batch_size - 1) // batch_size

        for batch_idx in range(num_batches):
            start = batch_idx * batch_size
            end = start + batch_size
            batch_df = df_tmp.iloc[start:end]
            
            # 檔名格式：folder_1.csv、folder_2.csv...
            output_file = os.path.join(output_dir, f'{folder}_{batch_idx + 1}.csv')
            batch_df.to_csv(output_file, index=False)
            print(f'{output_file}:{batch_df.shape[0]}')

In [10]:
def delete_files_except(csv_path, images_folder):
    try:
        # 讀取 CSV 文件，假設沒有標題列
        files_to_keep = pd.read_csv(csv_path, header=None)[0].tolist()
        
        # 計數器
        deleted_count = 0
        kept_count = 0
        
        # 遍歷資料夾中的所有檔案
        for filename in os.listdir(images_folder):
            if filename.endswith('.webp'):  # 只處理 .webp 檔案
                file_path = os.path.join(images_folder, filename)
                
                if filename in files_to_keep:
                    kept_count += 1
                    print(f"保留: {filename}")
                else:
                    try:
                        os.remove(file_path)
                        deleted_count += 1
                        print(f"刪除: {filename}")
                    except Exception as e:
                        print(f"刪除 {filename} 時發生錯誤: {str(e)}")
        
        print(f"\n處理完成！")
        print(f"保留的圖片數量: {kept_count}")
        print(f"刪除的圖片數量: {deleted_count}")
        
    except Exception as e:
        print(f"發生錯誤: {str(e)}")

---

In [66]:
root_folder = './images/check'
foldernames = list_all_folders(root_folder)
foldernames

['20240509_LingOrmCh3FanGreet']

In [67]:
# foldernames = ['check']
for foldername in foldernames:
    df = find_jpg_files(root_folder, foldername)
    data_to_csv(df)


20240509_LingOrmCh3FanGreet: (199, 3)
./data/test\20240509_LingOrmCh3FanGreet_1.csv:199


### 刪除photo

In [68]:
path = './images/check'
files = '''
bunnytales1221_1789231834712420508_2024-05-11 09-51_1.webp
EEgg17559_1788457665699692832_2024-05-09 06-35_1.webp
butterflyverse__1788617988708630979_2024-05-09 17-12_4.webp
mtaengoo_1789163087347786050_2024-05-11 05-18_4.webp
teddybear_t9_1788473243214770663_2024-05-09 07-37_2.webp
LOgallery38_1788491102372438017_2024-05-09 08-48_3.webp
GladtoseeU_1127_1789978243203825909_2024-05-13 11-17_1.webp
LOgallery38_1788721448422711336_2024-05-10 00-03_1.webp
LOgallery38_1788722609154335010_2024-05-10 00-07_4.webp
Abears19_1788541517227069717_2024-05-09 12-08_2.webp
GladtoseeU_1127_1789979161034924097_2024-05-13 11-21_1.webp
mtaengoo_1788512149087858699_2024-05-09 10-11_3.webp
mtaengoo_1790596626303852724_2024-05-15 04-14_2.webp
ormstagram_1789950434880155915_2024-05-13 09-26_3.webp
rushhour39_1788512527623819443_2024-05-09 10-13_2.webp
LOgallery38_1788722609154335010_2024-05-10 00-07_2.webp
mtaengoo_1830788409662808397_2024-09-03 02-02_1.webp
bunnytales1221_1789148583578755101_2024-05-11 04-20_2.webp
LOgallery38_1788550590743876046_2024-05-09 12-44_1.webp
rushhour39_1788469367547830319_2024-05-09 07-21_4.webp
Abears19_1788554453035176162_2024-05-09 12-59_2.webp
ormstagram_1789950434880155915_2024-05-13 09-26_2.webp
LOgallery38_1788470811327877593_2024-05-09 07-27_3.webp
bunnytales1221_1807621957946581348_2024-07-01 03-47_1.webp
_mars09__1788504590272577801_2024-05-09 09-41_3.webp
butterflyverse__1790186369165513117_2024-05-14 01-04_1.webp
'''

In [69]:
delete_file(files, path)

成功刪除 26 個檔案
刪除失敗 0 個檔案


### 保留photo

In [11]:
delete_files_except('files_to_preserve.csv', './images/check/test')

刪除: linglingkwong_2207453439323920565_2019-12-26 14-02_1.webp
保留: linglingkwong_2212199552031650398_2020-01-02 03-12_1.webp
保留: linglingkwong_2265145626769354018_2020-03-15 04-27_1.webp
刪除: linglingkwong_2311425267079246523_2020-05-18 00-56_1.webp
刪除: linglingkwong_2311425267079246523_2020-05-18 00-56_2.webp

處理完成！
保留的圖片數量: 2
刪除的圖片數量: 3


---

In [2]:
df_no_dup = pd.read_excel('photo_data.xlsx')
df_no_dup['filename'] = df_no_dup['filename'].apply(lambda x:x.replace('.jpg', '.webp'))
df_no_dup = df_no_dup[(df_no_dup['root_path']=='雜誌') | (df_no_dup['root_path']=='拍攝')]
df2 = pd.read_csv('./data/images_活動.csv')

In [3]:
df_no_dup.shape

(1229, 10)

In [4]:
df_no_dup.root_path.value_counts()

root_path
雜誌    617
拍攝    612
Name: count, dtype: int64

In [5]:
df2.shape

(5523, 2)

In [6]:
df2.head(2)

,filename,full_path
0,muimui_za_1831621706043289845_2024-09-05 09-13...,./images/活動/activity/20231101_試鏡/muimui_za_183...
1,muimui_za_1831621706043289845_2024-09-05 09-13...,./images/活動/activity/20231101_試鏡/muimui_za_183...


In [7]:
df_no_dup.head(2)

,filename,full_path,username,relative_path,file_date,file_size,extension,root_path,children_path,month_path
2850,cocochicwedding_3485939477002385180_2024-10-24...,/Users/linnianyi/Library/CloudStorage/GoogleDr...,cocochicwedding,/雜誌/Bride雜誌,2024-10-24,706477,jpg,雜誌,Bride雜誌,NaN
2851,PP_0566_1819186007704850790_2024-08-02 01-38_1...,/Users/linnianyi/Library/CloudStorage/GoogleDr...,PP_0566,/雜誌/Bride雜誌,2024-08-02,405683,jpg,雜誌,Bride雜誌,NaN


In [8]:
df_no_dup1 = df_no_dup[['filename', 'root_path']]

In [9]:
df_no_dup1.head(2)

,filename,root_path
2850,cocochicwedding_3485939477002385180_2024-10-24...,雜誌
2851,PP_0566_1819186007704850790_2024-08-02 01-38_1...,雜誌


In [10]:
df2 = df2.merge(df_no_dup1, on='filename', how='left')

In [11]:
df2.shape

(5523, 3)

In [12]:
df2.head()

,filename,full_path,root_path
0,muimui_za_1831621706043289845_2024-09-05 09-13...,./images/活動/activity/20231101_試鏡/muimui_za_183...,NaN
1,muimui_za_1831621706043289845_2024-09-05 09-13...,./images/活動/activity/20231101_試鏡/muimui_za_183...,NaN
2,muimui_za_1831621706043289845_2024-09-05 09-13...,./images/活動/activity/20231101_試鏡/muimui_za_183...,NaN
3,muimui_za_1831621706043289845_2024-09-05 09-13...,./images/活動/activity/20231101_試鏡/muimui_za_183...,NaN
4,muimui_za_1831900664223674858_2024-09-06 03-42...,./images/活動/activity/20231101_試鏡/muimui_za_183...,NaN


In [13]:
df2['root_path'].value_counts()

root_path
拍攝    25
Name: count, dtype: int64

In [18]:
aa = df2[df2['root_path']=='拍攝']

In [19]:
aa.head()

,filename,full_path,root_path
888,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
889,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
890,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
891,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
892,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝


In [20]:
aa

,filename,full_path,root_path
888,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
889,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
890,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
891,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
892,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
893,fabrique.co_3451107824765875200_2024-09-06 12-...,./images/活動/notion/fabrique.co_345110782476587...,拍攝
1989,monday_21_5075378859416603_2024-09-05 14-54_2....,./images/活動/notion/monday_21_5075378859416603_...,拍攝
1990,monday_21_5075378859416603_2024-09-05 14-54_4....,./images/活動/notion/monday_21_5075378859416603_...,拍攝
2213,orm.kornnaphat_3447105999466233263_2024-08-31 ...,./images/活動/notion/orm.kornnaphat_344710599946...,拍攝
2214,orm.kornnaphat_3449685543620646636_2024-09-04 ...,./images/活動/notion/orm.kornnaphat_344968554362...,拍攝


In [21]:
aa.shape

(25, 3)

In [22]:
for index, row in aa.iterrows():
    image_path = row['full_path']
    try:
        shutil.copy(image_path, './del')
    except Exception as e:
        print(image_path, str(e))

In [23]:
success_count = 0
failure_count = 0
failed_files = []
for index, row in aa.iterrows():
    image_path = row['full_path']
    try:
        os.remove(image_path)
        success_count += 1
    except Exception as e:
        failure_count += 1
        failed_files.append((image_path, str(e)))

In [24]:
print(f"成功刪除 {success_count} 個檔案")
print(f"刪除失敗 {failure_count} 個檔案")

成功刪除 25 個檔案
刪除失敗 0 個檔案
